# 接口：从协议到抽象基类

In [ ]:
# 不建议自己编写抽象基类，因为很容易过度设计
# 协议是接口，但不是正式的

In [1]:
from random import shuffle
l = list(range(10))
shuffle(l)
print (l)

[0, 1, 9, 6, 3, 5, 8, 2, 4, 7]


In [5]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [6]:
class FrenchDeck:
    ranks =  [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self.cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self.cards)

    # 实现 getitem方法，就可以迭代
    # xxx in FrenchDeck，若没有__contains__则做一次迭代搜索
    def __getitem__(self, position):
        return self.cards[position]

In [7]:
deck = FrenchDeck()
print (len(deck))

52


In [10]:
def set_card(deck, position, card):
    deck.cards[position] = card
# 猴子补丁
FrenchDeck.__setitem__ = set_card
shuffle(deck)
print (deck[:5])

[Card(rank='J', suit='diamonds'), Card(rank='K', suit='hearts'), Card(rank='9', suit='diamonds'), Card(rank='8', suit='spades'), Card(rank='10', suit='spades')]


In [ ]:
# 实例化该类时，会检查有没有实现抽象方法，没实现的话，会报错
class FrenchDeck2(collections.MutableSequence):
    ranks =  [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self.cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self.cards)

    def __getitem__(self, position):
        return self.cards[position]

    def __setitem__(self, position, value):
        self.cards[position] = value

    def __delitem__(self, position):
        del sel.cards[position]

    def insert(self, position, value):
        self.cards.insert(position, value)

In [ ]:
# 集合抽象基类
# collections.abc
# abc.ABC 每个抽象基类都依赖这个类，不用导入它，除非定义新的抽象基类
# Iterable、Container 和 Sized
# Sequence、Container 和 Set
# MappingView
# Callable 和 Hashtable
# Iterator

In [ ]:
# 标准库中最有用的抽象基类包是numbers
# Number、Complex、Real、Rational、Integral
# isinstance(x, numbers.Integral)
# isinstance(x, numbers.Real)

# 定义并使用一个抽象基类

In [2]:
import abc

In [3]:
# 抽象方法可以有实现代码，即便实现了，子类也必须覆盖抽象方法，但是在子类中可以使用super()函数调用抽象方法
class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""

    @abc.abstractmethod
    def pick(self):
        """随机删除元素，然后将其返回，如果实例为空，这个方法应该抛出 LookupError"""

    def loaded(self):
        """如果至少有一个元素，返回True，否则返回False"""
        return bool(self.inspect())

    def inspect(self):
        """返回一个有序元祖，由当前元素构成"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [4]:
# 用一个有缺陷的类糊弄Tombola
class Fake(Tombola):
    def pick(self):
        return l3
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [ ]:
# @abstractmethod
# @abstractclassmethod
# @abstractstaticmethod
# 可以用装饰器堆叠
# @classmethod
# @abstractmethod

In [6]:
# 定义Tombola抽象基类的子类
import random

In [ ]:
class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [ ]:
class LotteryBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        tyr:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(sorted(self._balls))

In [ ]:
# Tombola 的虚拟子类
@Tombola.register
class TomboList(list):
    def pick(self):
        if self:
            position = random.randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend
    
    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(sorted(self))

In [10]:
class Struggle:
    def __len__(self): return 23
from collections import abc
print (isinstance(Struggle(), abc.Sized))
print (issubclass(Struggle, abc.Sized))

True
True


In [ ]:
# 子类检查 issubclass 用 def __subclass__(cls, C)